In [0]:
from pyspark.sql.functions import to_date, col, lit, month, date_format, lower
data = spark.read.csv('dbfs:/FileStore/tables/crimes_2017.csv', header=True, inferSchema=True).withColumn('Date', to_date(col('Date'), 'MM/dd/yyyy hh:mm:ss a'))

# Lab #4 PySpark DataFrame 

- Once you have the dataframe loaded: 
    - display the first 5 rows. Including: 
        - the code which loaded the dataframe 
        - which transformations are being performed and,
        - which actions are being performed? 

In [0]:
data.show(5)
# spark.read.csv is a transformation which returns a dataframe from the csv file
# withcolumn is an action which creates a column 'date' from the date column in the csv using the MM/dd/yyyy hh:mm:ss a format

+--------+-----------+----------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|      Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+----------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11227634|   JB147599|2017-08-26| 001XX W RANDOLPH ST|0281|CRIM SEXUAL ASSAULT|      NON-AGGRAVATED|         HOTEL/MOTEL| false|   false| 122|       1|  42|            32|      02|        null|        null|2017|02/11/

- Output the top 5 dates for reported crimes and the number of crimes on each day.  
    - Order in descending order by the number of crimes reported. Lab Exercise part 3 (4 marks)


In [0]:
top5Dates = data.groupBy('Date').count().sort('count', ascending=False)
top5Dates.show(5)

+----------+-----+
|      Date|count|
+----------+-----+
|2017-01-01| 1256|
|2017-08-01|  965|
|2017-07-01|  937|
|2017-08-05|  928|
|2017-08-04|  926|
+----------+-----+
only showing top 5 rows



    - Output the top 3 crimes reported and the number of times each was reported, for the day with the most number of reported crimes from part 3.

In [0]:
top3Crimes = (data.filter(col("Date") == top5Dates.first()["Date"])
.groupBy("Primary Type")
.count()
.sort('count', ascending=False))
top3Crimes.show(3)

+------------------+-----+
|      Primary Type|count|
+------------------+-----+
|DECEPTIVE PRACTICE|  208|
|           BATTERY|  207|
|             THEFT|  187|
+------------------+-----+
only showing top 3 rows



- Which month of the year has the most number of crimes. 
- Output the month and the number of crimes.

In [0]:
highestCrimeVolumeByMonth = (
    data.withColumn("Month", date_format(col("date"), "MMMM"))
                            .groupBy("Month")
                            .count()
                            .sort('count', ascending=False)
                            )
highestCrimeVolumeByMonth.show(1)

+-----+-----+
|Month|count|
+-----+-----+
| July|24889|
+-----+-----+
only showing top 1 row



- Filter the dataframe to create a new dataframe which only contains
crimes where the word '**gun**' appears in the *'Description'* column. 
- What percentage of crimes involve a gun? Output all the unique ‘Description’ values which include ‘gun’.

In [0]:
onlyCrimesWithTheWordGun = data.filter(lower(col("Description")).contains("gun"))
percentageOfGunCrimes = (onlyCrimesWithTheWordGun.count() / data.count())
print(str(round(percentageOfGunCrimes * 100, 2)) + "% of crimes involve a gun")
onlyCrimesWithTheWordGun.select("Description").distinct().show(truncate = False)

5.38% of crimes involve a gun
+--------------------------------------------------+
|Description                                       |
+--------------------------------------------------+
|AGGRAVATED POLICE OFFICER - HANDGUN               |
|UNLAWFUL SALE HANDGUN                             |
|UNLAWFUL POSS OF HANDGUN                          |
|ATTEMPT ARMED - HANDGUN                           |
|ARMED - HANDGUN                                   |
|AGGRAVATED: HANDGUN                               |
|GUN OFFENDER: DUTY TO REGISTER                    |
|ARMED: HANDGUN                                    |
|AGGRAVATED - HANDGUN                              |
|UNLAWFUL USE HANDGUN                              |
|AGGRAVATED PO: HANDGUN                            |
|GUN OFFENDER NOTIFICATION-NO CONTACT              |
|ATTEMPT: ARMED-HANDGUN                            |
|GUN OFFENDER: ANNUAL REGISTRATION                 |
|AGGRAVATED DOMESTIC BATTERY: HANDGUN              |
|AGG PRO.EMP: HA